In [7]:
import numpy as np
import pandas as pd
import re

In [10]:
# import data from txt
with open("2024\Day_02\data.txt", "r") as f:
    data = f.read().strip().split("\n")
int_data = [list(map(int, re.findall("\\d+", x))) for x in data]

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Madz\AppData\Local\Temp\ipykernel_21548\657662112.py:2: SyntaxWarning: invalid escape sequence '\D'
  with open("2024\Day_02\data.txt", "r") as f:


In [ ]:
# Day 2 help from Reddit...
def safe1(levels):
    return all(1 <= abs(n1 - n2) <= 3 for n1, n2 in zip(levels, levels[1:])) and (
        levels == sorted(levels) or levels == sorted(levels)[::-1]
    )


def safe2(levels):
    return any(safe1(levels[:i] + levels[i + 1 :]) for i in range(len(levels)))


# Part 1
print(sum(safe1(levels) for levels in int_data))

# Part 2
print(sum(safe2(levels) for levels in int_data))

390
439
